## BioMistral Medical RAG Chatbot using BioMistral Open Source LLM

In [6]:
import warnings
from pydantic import BaseModel

warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

In [1]:
!pip install langchain sentence-transformers chromadb llama-cpp-python pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8

In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00


In [12]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

In [14]:
loder = PyPDFLoader("https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf")
docs = loder.load()

In [15]:
len(docs)

95

In [16]:
docs[23]

Document(metadata={'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf', 'page': 23}, page_content='19Major Risk FactorsGetting Tested\nHigh blood cholesterol itself does not cause symptoms, so if your cho-lesterol level is too high, you may not be aware of it. That’s why it isimportant to get your cholesterol levels checked regularly. Starting atage 20, everyone should have their cholesterol levels checked bymeans of a blood test called a lipoprotein profile. Be sure to ask forthe test results, so you will know whether you need to lower yourcholesterol. Ask your doctor how soon you should be retested.\nTotal cholesterol is a measure of the cholesterol in all of your\nlipoproteins, including the “bad” cholesterol in LDL and the “good”cholesterol in HDL. An LDL level below 100 mg/dL is considered“optimal” or ideal. As you can see in the accompanying table, thereare four other categories of LDL levels. The higher your LDL num-ber, the higher your risk of heart d

## Chunking

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks=text_splitter.split_documents(docs)

In [18]:
len(chunks)

747

In [20]:
chunks[680]

Document(metadata={'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf', 'page': 84}, page_content='you may experience temporary withdrawal symptoms such as headaches, irritability, tiredness, and trouble concentrating.While these feelings are not pleasant, it may help to know that they are signs that your body is recovering from smoking. Mostsymptoms end within 2 to 4 weeks.')

In [22]:
chunks[746]

Document(metadata={'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf', 'page': 94}, page_content='U.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of HealthNational Heart, Lung, and Blood Institute\nNIH Publication No. 06–5269\nNovember 2005ISBN 1-933236-03-5')

## Embedding Creation

In [23]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]="Huggigfacehub_api_token"

In [25]:
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

## Vector store creation

In [26]:
vectorstore = Chroma.from_documents(chunks,embeddings)

In [27]:
query = "Who is at risk of heart disease?"

search_results = vectorstore.similarity_search(query)

In [28]:
search_results

[Document(metadata={'page': 8, 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 6, 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf'}, page_content='factor greatly increases the chances of developing heart disease.Moreover, the worse a particular risk factor is, the more likely youare to develop heart disease. For example, if you have high bloodpressure, the higher it is, the greater your chan

In [30]:
#Give Top 5 matches results
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [31]:
retriever.get_relevant_documents(query)

<ipython-input-31-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'page': 8, 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 6, 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf'}, page_content='factor greatly increases the chances of developing heart disease.Moreover, the worse a particular risk factor is, the more likely youare to develop heart disease. For example, if you have high bloodpressure, the higher it is, the greater your chan

## LLM Model Loading

In [33]:
!pip install huggingface_hub
from huggingface_hub import hf_hub_download
import os

# Download the model file
repo_id = "MaziyarPanahi/BioMistral-7B-GGUF"
filename = "BioMistral-7B.Q4_K_M.gguf"
cache_dir = "biomistral_model" # create a directory to store the model

model_path = hf_hub_download(repo_id=repo_id, filename=filename, cache_dir=cache_dir)

# Load the model using the local file path
from langchain.llms import LlamaCpp
llm = LlamaCpp(
    model_path = model_path,
    temperature = 0.2,
    max_tokens = 2048,
    top_p = 1,
)



BioMistral-7B.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from biomistral_model/models--MaziyarPanahi--BioMistral-7B-GGUF/snapshots/ae4f07544f1015dc8f5bf382b7582852638cbecf/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u3

## Use LLM and Retriever and Query to generate the final response

In [34]:
template = """
<|context|>
You are an medical assistant that follows the instructions and generate the accurate response based on query and the context provided.
please be truthful and give the answers.
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [40]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [42]:
# Define the prompt template
prompt = ChatPromptTemplate.from_messages([("user",template)])

In [44]:
rag_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [45]:
response = rag_chain.invoke(query)


llama_print_timings:        load time =    3031.05 ms
llama_print_timings:      sample time =      26.41 ms /    49 runs   (    0.54 ms per token,  1855.29 tokens per second)
llama_print_timings: prompt eval time =   28856.19 ms /    72 tokens (  400.78 ms per token,     2.50 tokens per second)
llama_print_timings:        eval time =   30630.73 ms /    49 runs   (  625.12 ms per token,     1.60 tokens per second)
llama_print_timings:       total time =   59598.87 ms /   121 tokens


In [46]:
response

'The people who are overweight, have a family history of heart disease, or smoke are at higher risk for heart disease. People with diabetes, high blood pressure, and high cholesterol also have an increased risk of heart disease.'

In [ ]:
# with user input
import sys

while True:
  user_input = input(f"Enter Your Query : ")
  if user_input == "exit":
    sys.exit()
  if user_input == "":
    continue
  result = rag_chain.invoke(user_input)
  print("\nAnswer : ", result)

Enter Your Query : How High blood cholesterol affect heart health?


Llama.generate: 54 prefix-match hit, remaining 22 prompt tokens to eval

llama_print_timings:        load time =    3031.05 ms
llama_print_timings:      sample time =      43.22 ms /    78 runs   (    0.55 ms per token,  1804.55 tokens per second)
llama_print_timings: prompt eval time =    7741.24 ms /    22 tokens (  351.87 ms per token,     2.84 tokens per second)
llama_print_timings:        eval time =   46391.27 ms /    77 runs   (  602.48 ms per token,     1.66 tokens per second)
llama_print_timings:       total time =   54236.78 ms /    99 tokens



Answer :  High blood cholesterol can cause plaque to form in your artery walls, which narrows the arteries and makes it harder for blood to flow. This increases your risk of heart disease and stroke. It is important to manage your cholesterol levels by eating a healthy diet, exercising regularly, maintaining a healthy weight, and taking medication as prescribed.
Enter Your Query : How smoke affects heart health?


Llama.generate: 55 prefix-match hit, remaining 16 prompt tokens to eval

llama_print_timings:        load time =    3031.05 ms
llama_print_timings:      sample time =      81.40 ms /   142 runs   (    0.57 ms per token,  1744.47 tokens per second)
llama_print_timings: prompt eval time =    5606.69 ms /    16 tokens (  350.42 ms per token,     2.85 tokens per second)
llama_print_timings:        eval time =   84158.15 ms /   141 runs   (  596.87 ms per token,     1.68 tokens per second)
llama_print_timings:       total time =   89961.37 ms /   157 tokens



Answer :  Smoking can affect your heart in many ways. Cigarette smoking can cause: 1) Your arteries become narrower, which slows blood flow to your heart and increases your blood pressure. 2) Smoke causes a buildup of fatty deposits within your arteries, called plaques, that may lead to coronary heart disease (CHD). 3) Smoking can cause arrhythmias, or irregular heartbeats, which can increase the risk of sudden cardiac death. 4) Smoking can also increase the risk of blood clots forming in your arteries, which can block blood flow and lead to a heart attack.
